In [1]:
import ee
import geemap
import logging
from filecmp import dircmp
import os
import filecmp
from dotenv import load_dotenv
from intake import open_catalog
import matplotlib as plt
import pandas as pd
import numpy as np
from shapely import wkt
import shapely
from sqlalchemy import create_engine, text
import geopandas as gpd
from dotenv import load_dotenv
load_dotenv()
import ipywidgets as widgets
from datetime import datetime, date, timedelta
from ipywidgets import HBox, VBox
import uuid
from datetime import datetime
import math
from ipywidgets import Button, Layout
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import asyncio

In [2]:
auth_file = '/home/oriane/test_app/gee_credentials.json'
os.environ['EARTHENGINE_TOKEN'] = auth_file

In [5]:
#ee.Authenticate()


Successfully saved authorization token.


In [6]:
ee.Initialize(project='ee-surfor')

In [7]:
### Variables
table_source = "sentinel_surfaces_detectees"
table_faits = "faits_zae_sentinel_surfaces_detectees"
table_information = "sentinel_informations_surfaces_detectees"
table_temp_cq = "liste_surfaces_cq_temp"
filter_list=['ZAE','Autre','CQ Brute','CQ PI']
filter_list_sample=['Nouveau','Existant']
mode_list=["Choix",'PI par groupe', 'CQ', 'PI FLAG','PI Entrainement']
filter_visual_check=['Polygone PI','Autre']
day_collection_init=60
catalog_path = f'{os.getenv("DATA_CATALOG_DIR")}/data_reference_feux.yaml'

In [ ]:
import ee
import geemap
import pandas as pd
import ipywidgets as widgets
from datetime import timedelta, datetime
from IPython.display import display
import concurrent.futures
def map_VC():
    upper_left_map = geemap.Map(center=(-21.2, 166), zoom=8, layout={'height': '500px', 'width': '100%'})
    upper_right_map = geemap.Map(center=(-21.2, 166), zoom=8, layout={'height': '500px', 'width': '100%'})
    lower_left_map = geemap.Map(center=(-21.2, 166), zoom=8, layout={'height': '500px', 'width': '100%'})
    lower_right_map = geemap.Map(center=(-21.2, 166), zoom=8, layout={'height': '500px', 'width': '100%'})

    polygone_id_widget = widgets.Text(value='L2A_T58KEB_20230612_8f9f515516da860', description='Polygon ID:', disabled=False)
    date_detection = widgets.Label(value='Détection: ')
    
    choix_interval = widgets.FloatText(value=60, description='Interval:', disabled=False)
    image_viewer = widgets.Play(interval=5000, value=0, min=0, max=1, step=1, description="Images", disabled=True)
    date_label = widgets.Label(value='Date: ')
    slider = widgets.IntSlider(step=1, value=0)

    def load_gdf(polygone_id):
        sql = f"""SELECT row_number() OVER () AS id,
        si.date_,
        si.surface_id_h3,
        ss.her,
        si.geometry
        FROM feux_cq.{table_source} si
        LEFT JOIN feux_cq.{table_information} ss ON ss.surface_id_h3 = si.surface_id_h3
        WHERE si.surface_id_h3 = '{polygone_id}'
        """

        catalog = open_catalog(catalog_path)
        dataCatalog = getattr(catalog, table_source)(sql_expr=sql)
        gdf = dataCatalog.read()

        if gdf.duplicated(subset=['surface_id_h3']).sum() >= 1:
            gdf = gdf.drop_duplicates(subset=['surface_id_h3'])

        gdf['date_'] = pd.to_datetime(gdf['date_'])
        gdf = gdf.to_crs(epsg=4326)
        
        return gdf

    def display_images(selected_gdf, interval):
        index = 0
        detection_date = selected_gdf.iloc[index]['date_'].strftime('%Y-%m-%d')
        date_detection.value = f'Date Détection : {detection_date}'
        date = selected_gdf.iloc[index]['date_'] - timedelta(days=interval)
        end_date = selected_gdf.iloc[index]['date_'] + timedelta(days=interval)
        geom = selected_gdf.iloc[index]['geometry']
        geom_json = geom.__geo_interface__
        ee_geom = ee.Geometry(geom_json)
        buffered_geom = ee_geom.buffer(3000)
        asset = ee.FeatureCollection(ee_geom)
            
        image_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
            .filterBounds(buffered_geom.bounds()) \
            .filterDate(date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 70))
        
        image_fix = image_collection.filterDate(selected_gdf.iloc[index]['date_'], selected_gdf.iloc[index]['date_'] + timedelta(hours=24)).median()
        image_fix = image_fix.clip(buffered_geom.bounds())

        vis_params_polygon = {
            'color': '000000', 
            'pointSize': 3,
            'pointShape': 'circle',
            'width': 1.5,
            'lineType': 'solid',
            'fillColor': '00000000',
        }
        rgb_params = {
            'bands': ['B4', 'B3', 'B2'],
            'min': 0,
            'max': 3000,
            'gamma': 1.4
        }
        fcir_params = {
            'bands': ['B8', 'B4', 'B3'],
            'min': 0,
            'max': 3000,
            'gamma': 1.4
        }

        image = image_fix
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
        image = image.addBands(ndvi)
        vis_params_ndvi = {
            'bands': ['ndvi'],
            'min': 0,
            'max': 1,
            'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
                        '74A901', '66A000', '529400', '3E8601', '207401', '056201',
                        '004C00', '023B01', '012E01', '011D01', '011301']
        }

        def load_and_display_map(map_obj,image, vis_params, label):
            map_obj.addLayer(image, vis_params, label)
            map_obj.addLayer(asset.style(**vis_params_polygon), {}, 'Polygon')
            map_obj.centerObject(ee_geom, 17)

        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = []
            futures.append(executor.submit(load_and_display_map, lower_right_map,image_fix, rgb_params, 'RGB'))
            futures.append(executor.submit(load_and_display_map, lower_left_map,image_fix, fcir_params, 'FCIR'))
            futures.append(executor.submit(load_and_display_map, upper_right_map,image, vis_params_ndvi, 'NDVI'))

            for future in concurrent.futures.as_completed(futures):
                future.result()

        dates_list = image_collection.aggregate_array('system:time_start').getInfo()
        dates_list = [datetime.utcfromtimestamp(date / 1000).strftime('%Y-%m-%d') for date in dates_list]

        slider.max = len(dates_list) - 1
        image_viewer.max = len(dates_list) - 1
        image_viewer.value = 0
        image_viewer.min = 0
        image_viewer.disabled = (len(dates_list) == 0)
        image_viewer.unobserve_all()

        def start_image(index):
            image = ee.Image(image_collection.toList(image_collection.size()).get(index))
            image = image.clip(buffered_geom.bounds())
        
            upper_left_map.addLayer(image, rgb_params, 'RGB')
            upper_left_map.addLayer(asset.style(**vis_params_polygon), {}, 'Polygon')
            upper_left_map.centerObject(ee_geom, 17)
            date_label.value = f'Date Image : {dates_list[index]}'

        widgets.jslink((image_viewer, 'value'), (slider, 'value'))
        widgets.interact(start_image, index=image_viewer)

    def on_value_change(change):
        gdf = load_gdf(polygone_id_widget.value)
        display_images(gdf, choix_interval.value)

    polygone_id_widget.observe(on_value_change, names='value')
    choix_interval.observe(on_value_change, names='value')

    gdf = load_gdf(polygone_id_widget.value)
    display_images(gdf, choix_interval.value)

    widgets_info = widgets.VBox([widgets.HBox([polygone_id_widget, date_detection]), choix_interval, widgets.HBox([image_viewer, date_label, slider])])
    widgets_map = widgets.VBox([widgets.HBox([upper_left_map, upper_right_map]), widgets.HBox([lower_left_map, lower_right_map])])
    display(widgets_info, widgets_map)

map_VC()

interactive(children=(Play(value=0, description='Images', interval=5000, max=35), Output()), _dom_classes=('wi…

interactive(children=(Play(value=0, description='Images', interval=5000, max=34), Output()), _dom_classes=('wi…

No such comm: 1c6f919cdf4346ac965e360652da7c1b
No such comm: 1c6f919cdf4346ac965e360652da7c1b
No such comm: 08671490a33744d9a062fcbdbfe17c05
No such comm: 08671490a33744d9a062fcbdbfe17c05
No such comm: 375f5dbd8cef48de808e68e3ced9ad45
No such comm: 375f5dbd8cef48de808e68e3ced9ad45
No such comm: 72f9d76130a644b3902253ea6f409a07
No such comm: 72f9d76130a644b3902253ea6f409a07
No such comm: a3be16adc4af4c9494ecc33ac0cea062
No such comm: a3be16adc4af4c9494ecc33ac0cea062
No such comm: 1804b12c2ce545869a7614fa05aff6d9
No such comm: 1804b12c2ce545869a7614fa05aff6d9
